## Libraries

In [54]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import en

## Read input file

## Train algorithm

In [55]:
#Training the algorithm
#To-do: Instead of speech train it on actual test case corpus
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
train_text = state_union.raw("2005-GWBush.txt")
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

## fn: Rule engine for multiple 'should' statement

In [56]:
def transform_stmt_mul_shd(stmt):
    temp_pst_collate = ""
    splt_sentence = stmt.split("and")
    for stmt in splt_sentence:
        temp_pst_collate = temp_pst_collate + 'and' + transform_stmt_single_shd(stmt)
        #print(temp_pst_collate)
    return temp_pst_collate[3:]          

## fn: Rule engine for single 'should' statement

In [57]:
def transform_stmt_single_shd(stmt): 
    words = nltk.word_tokenize(stmt)
    veb_lst = [word for (word, pos) in nltk.pos_tag(words) if is_verb(pos)]
    
    if (stmt.find("should not") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should not be", "is not")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("details") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("properties") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("fields") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("sections") > -1):
        trnf_pst_stmt = stmt.replace("should be", "are")
        return trnf_pst_stmt
         
    elif (stmt.find("should") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should be", "is")
        return trnf_pst_stmt
        
    elif (stmt.find("should") > -1) and ('be' not in veb_lst):
        trnf_pst_stmt = stmt.replace("should "+veb_lst[0], en.verb.present(veb_lst[0].lower()))
        return trnf_pst_stmt
    
    else:
        return stmt

In [58]:
# creating a function that will retrive only base verb forms
is_verb = lambda pos: pos == 'VB' 

In [59]:
#verb_lst = []
all_past_stmt = []
temp_pst = ""

for test_case in list(df["Expect_res"]):
    stmt_lst = custom_sent_tokenizer.tokenize(test_case)
    try:
        for stmt in stmt_lst:
            words = nltk.word_tokenize(stmt)
            if words.count("should") > 1:
                temp_pst = temp_pst + '\n' + transform_stmt_mul_shd(stmt)
                
            else:
                temp_pst = temp_pst + '\n' + transform_stmt_single_shd(stmt)
                
        all_past_stmt.append(temp_pst[1:])
        #print (temp_pst)
        temp_pst = ""

    except Exception as e:
        all_past_stmt.append(test_case)
        print(str(e))

## Convert Actual Result - Cut [user: ]

In [60]:
trm_act_res = []
for test_case in list(df["actualresult"]):
    try:
        #print (test_case)
        trm_act_res.append(test_case[:test_case.rfind("[")])
    except Exception as e:
        trm_act_res.append(test_case)
        #print(str(e))   

## Write "Converted Expected" & "Converted Actual" to file

In [61]:
se = pd.Series(all_past_stmt)

In [62]:
ac = pd.Series(trm_act_res)

In [63]:
df['Converted_Expected'] = se.values

In [64]:
df['Converted_Actual'] = ac.values

In [65]:
#returns the dimesnsion of the data
df.shape

(931, 6)

In [66]:
#returns number of lines in the list
len(se)

931

In [67]:
# display dimension of the df after adding out column
df.shape

(931, 6)

In [68]:
#df['out']

## Create new output file

In [69]:
#print(str.replace(' ', ''))

In [70]:
'''str2 = "Testing the comparison between actual and expected for final results"
(str1.replace(' ', '')) == (str2.replace(' ', ''))'''

'str2 = "Testing the comparison between actual and expected for final results"\n(str1.replace(\' \', \'\')) == (str2.replace(\' \', \'\'))'

In [71]:
df.shape[0]

931

## Compare "Converted expected" & "Converted Actual"

In [72]:
cmpexpactlst = []
resulttxt = ''
for index, row in df.iterrows():
    #print(row["Converted_Expected"] +"----> \n"+  row["Converted_Actual"])
    
    x = row["Converted_Expected"].replace(' ', '')
    x1 = x.replace('\n', '') #replacing new lines
    x2 = x1.replace('\r', '')#replacing returns
    
    y = row["Converted_Actual"].replace(' ', '')
    y1 = y.replace('\n', '') #replacing new lines
    y2 = y1.replace('\r', '') #replacing returnss
    
    if x2 == y2:
        resulttxt = 'Approved'
    elif x2 in y2:
        #y2.contains(x2):
        resulttxt = 'Approved'
    else:
        resulttxt = 'Verify'
    cmpexpactlst.append(resulttxt)

## Write comparison results to file

In [73]:
res = pd.Series(cmpexpactlst)

In [74]:
df["Compare_result"] = res.values

In [75]:
df.to_csv('Final_Output3.csv', encoding='utf-8')

In [76]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     print (words)
     return Counter(words)

text1 = 'System web the web'
text2 = 'System displays the web page'

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

cosine = get_cosine(vector1, vector2)

print 'Cosine:', cosine

['System', 'web', 'the', 'web']
['System', 'displays', 'the', 'web', 'page']
Cosine: 0.73029674334
